In [1]:
import nest_asyncio
nest_asyncio.apply()

import os
import time
import json
import pymysql
import asyncio
import aiohttp
import datetime
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('api_key')

request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

base_url = "https://kr.api.riotgames.com/tft/"
match_base_url = "https://asia.api.riotgames.com/tft/match/v1/"

In [2]:
# 챌린저 유저 데이터 가져오는 함수
async def fetch_challenger_data(session: aiohttp.ClientSession):
    url = base_url + 'league/v1/challenger'
    while True:
        async with session.get(url, headers=request_header) as response:
            if response.status == 429:  # Rate limit exceeded
                retry_after = int(response.headers.get('Retry-After', 1))
                await asyncio.sleep(retry_after)
                continue
            return await response.json()

In [3]:
# 매치 아이디 가져오는 함수
async def fetch_match_ids(session: aiohttp.ClientSession, puuid: str, count: int = 10):
    url = f"{match_base_url}matches/by-puuid/{puuid}/ids?count={count}"
    while True:
        async with session.get(url, headers=request_header) as response:
            if response.status == 429:
                retry_after = int(response.headers.get('Retry-After', 1))
                await asyncio.sleep(retry_after)
                continue
            if response.status == 200:
                return await response.json()

In [4]:
# 매치 상세 데이터 가져오는 함수
async def fetch_match_detail(session: aiohttp.ClientSession, match_id: str):
    url = f"{match_base_url}matches/{match_id}"
    while True:
        async with session.get(url, headers=request_header) as response:
            if response.status == 429:
                retry_after = int(response.headers.get('Retry-After', 1))
                await asyncio.sleep(retry_after)
                continue
            return await response.json()

In [5]:
# 챌린저 데이터 DataFrame 생성
async def process_challenger_data():
    async with aiohttp.ClientSession() as session:
        challenger  = await fetch_challenger_data(session)
        challenger_df = pd.DataFrame(challenger['entries'])
        challenger_df.to_csv('challenger_users.csv', index=False)
        return challenger_df

In [6]:
async def process_match_ids():
    async with aiohttp.ClientSession() as session:
        # 챌린저 데이터 가져오기
        challenger_df = await process_challenger_data()
        
        # 모든 매치 ID를 저장할 리스트
        all_match_ids = []
        
        # 병렬 처리를 위한 태스크 생성
        tasks = []
        for puuid in challenger_df['puuid'][:100]:
            tasks.append(fetch_match_ids(session, puuid))
        
        # 모든 태스크 동시 실행
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # 결과 처리
        for result in results:
            if isinstance(result, Exception):
                print(f"Error fetching matches: {str(result)}")
                continue
            all_match_ids.extend(result)
        
        # 중복 제거
        unique_match_ids = list(set(all_match_ids))
        
        # DataFrame 생성 및 저장
        match_ids_df = pd.DataFrame(unique_match_ids, columns=['match_id'])
        match_ids_df.to_csv('challenger_match_ids.csv', index=False)
        
        print(f"수집된 고유 매치 ID 개수: {len(unique_match_ids)}")
        return match_ids_df

In [7]:
async def process_match_details():
    async with aiohttp.ClientSession() as session:
        # 저장된 매치 ID 파일 읽기
        match_ids_df = pd.read_csv('challenger_match_ids.csv')
        match_ids = match_ids_df['match_id'].tolist()
        
        # 매치 상세 정보를 저장할 리스트
        match_details = []
        
        # 참가자 정보를 저장할 리스트
        participant_details = []

        # 병렬 처리를 위한 태스크 생성
        tasks = []
        for match_id in match_ids:
            tasks.append(fetch_match_detail(session, match_id))
        
        # 모든 태스크 동시 실행 (최대 50개씩)
        chunk_size = 50  # 청크 크기를 50으로 증가
        for i in range(0, len(tasks), chunk_size):
            chunk = tasks[i:i + chunk_size]
            results = await asyncio.gather(*chunk, return_exceptions=True)
            
            for match_id, result in zip(match_ids[i:i + chunk_size], results):
                if isinstance(result, Exception):
                    print(f"Error fetching match details for match_id {match_id}: {str(result)}")
                    continue
                
                try:
                    match_detail = result
                    # 필요한 정보 추출
                    match_info = {
                        'match_id': match_id,
                        'game_datetime': datetime.datetime.fromtimestamp(match_detail['info']['game_datetime'] / 1000),
                        'game_length': match_detail['info']['game_length'],
                        'game_version': match_detail['info']['game_version'],
                        'queue_id': match_detail['info']['queue_id'],
                        'tft_set_number': match_detail['info']['tft_set_number'],
                        #'tft_set_core_name': match_detail['info']['tft_set_core_name']
                    }
                    match_details.append(match_info)
                    
                    # 참가자 정보 추출
                    for participant in match_detail['info']['participants']:
                        participant_info = {
                            'match_id': match_id,
                            'puuid': participant['puuid'],
                            'placement': participant['placement'],
                            'level': participant['level'],
                            'gold_left': participant['gold_left'],
                            'last_round': participant['last_round'],
                            'players_eliminated': participant['players_eliminated'],
                            'total_damage_to_players': participant['total_damage_to_players'],
                            'traits': participant['traits'],
                            'units': participant['units']
                        }
                        participant_details.append(participant_info)
                    
                except Exception as e:
                    print(f"Error processing match details for match_id {match_id}: {str(e)}")
                    continue
            
            # API 요청 제한을 위한 대기 (0.5초로 단축)
            await asyncio.sleep(0.5)
            
            # 진행 상황 출력
            print(f"진행률: {min(i + chunk_size, len(match_ids))}/{len(match_ids)}")
        
        # DataFrame 생성 및 저장
        match_details_df = pd.DataFrame(match_details)
        match_details_df.to_csv('challenger_match_details.csv', index=False)
        
        # 참가자 정보 DataFrame 생성 및 저장
        participant_details_df = pd.DataFrame(participant_details)
        participant_details_df.to_csv('challenger_match_participants.csv', index=False)

        print(f"수집된 매치 상세 정보 개수: {len(match_details)}")
        print(f"수집된 매치 참가자 정보 개수: {len(participant_details)}")
        return match_details_df

In [8]:
async def main():
    start_time = time.time()
    
    # 챌린저 데이터 수집
    challenger_df = await process_challenger_data()
    print(f"챌린저 데이터 수집 완료: {time.time() - start_time:.2f}초")
    
    # 매치 ID 수집
    match_ids_df = await process_match_ids()
    print(f"매치 ID 수집 완료: {time.time() - start_time:.2f}초")
    
    # 매치 상세 정보 수집
    match_details_df = await process_match_details()
    print(f"매치 상세 정보 수집 완료: {time.time() - start_time:.2f}초")
    
    print(f"전체 실행 시간: {time.time() - start_time:.2f}초")

await main()

챌린저 데이터 수집 완료: 0.32초
수집된 고유 매치 ID 개수: 763
매치 ID 수집 완료: 9.36초
진행률: 50/763
진행률: 100/763
진행률: 150/763
진행률: 200/763
진행률: 250/763
진행률: 300/763
진행률: 350/763
진행률: 400/763
진행률: 450/763
진행률: 500/763
진행률: 550/763
진행률: 600/763
진행률: 650/763
진행률: 700/763
진행률: 750/763
진행률: 763/763
수집된 매치 상세 정보 개수: 763
수집된 매치 참가자 정보 개수: 6090
매치 상세 정보 수집 완료: 974.96초
전체 실행 시간: 974.96초


In [9]:
# 챌린저 유저 데이터 저장
conn = pymysql.connect(
    host='localhost',
    user='root',
    password=os.getenv('mysql_password'),
    db='TFT',
    charset='utf8'
)

cur = conn.cursor(pymysql.cursors.DictCursor)

cur.execute("select puuid from challenger_users")
db_users_df = pd.DataFrame(cur.fetchall())

challenger_user_df = pd.read_csv('challenger_users.csv')

# 강등당한 챌린저 유저 데이터 삭제
for index, row in db_users_df.iterrows():
    if row['puuid'] not in challenger_user_df['puuid'].values:
        query = "delete from challenger_users where puuid = %s"
        cur.execute(query, (row['puuid']))

# 승급한 챌린저 유저 데이터 저장
for index, row in challenger_user_df.iterrows():
    query = "insert ignore into challenger_users (summonerId, puuid, leaguePoints, `rank`, wins, losses) values (%s, %s, %s, %s, %s, %s) on duplicate key update leaguePoints = %s, `rank` = %s, wins = %s, losses = %s"
    cur.execute(query, (row['summonerId'],row['puuid'], row['leaguePoints'], row['rank'], row['wins'], row['losses'], row['leaguePoints'], row['rank'], row['wins'], row['losses']))

conn.commit()

In [10]:
# 매치 ID 저장
cur = conn.cursor(pymysql.cursors.DictCursor)

cur.execute("select match_id from challenger_match_details")
db_match_ids_df = pd.DataFrame(cur.fetchall())

match_ids_df = pd.read_csv('challenger_match_details.csv')

for index, row in db_match_ids_df.iterrows():
    if row['match_id'] not in match_ids_df['match_id'].values:
        query = "delete from challenger_match_details where match_id = %s"
        cur.execute(query, (row['match_id']))

for index, row in match_ids_df.iterrows():
    query = "insert ignore into challenger_match_details (match_id, game_datetime, game_length, game_version, queue_id, tft_set_number) values (%s, %s, %s, %s, %s, %s) on duplicate key update game_datetime = %s, game_length = %s, game_version = %s, queue_id = %s, tft_set_number = %s"
    cur.execute(query, (row['match_id'], row['game_datetime'], row['game_length'], row['game_version'], row['queue_id'], row['tft_set_number'], row['game_datetime'], row['game_length'], row['game_version'], row['queue_id'], row['tft_set_number']))
    
conn.commit()

In [11]:
cur = conn.cursor(pymysql.cursors.DictCursor)

#  DB 참가자 정보 파일 읽기
cur.execute("select puuid from challenger_match_participants")
db_match_participants_df = pd.DataFrame(cur.fetchall())

# CSV 참가자 정보 파일 읽기
match_participants_df = pd.read_csv('challenger_match_participants.csv')

# 참가자 정보 삭제
for index, row in db_match_participants_df.iterrows():
    if row['puuid'] not in match_participants_df['puuid'].values:
        query = "delete from challenger_match_participants where puuid = %s and match_id = %s"
        cur.execute(query, (row['puuid'], row['match_id']))

# 참가자 정보 저장
for index, row in match_participants_df.iterrows():
    query = "insert ignore into challenger_match_participants (match_id, puuid, placement, level, gold_left, last_round, players_eliminated, total_damage_to_players) values (%s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(query, (row['match_id'], row['puuid'], row['placement'], row['level'], row['gold_left'], row['last_round'], row['players_eliminated'], row['total_damage_to_players']))
conn.commit()

In [12]:
cur = conn.cursor(pymysql.cursors.DictCursor)

cur.execute("select match_id, puuid from challenger_match_participants")
db_traits_df = pd.DataFrame(cur.fetchall())

traits_df = pd.read_csv('challenger_match_participants.csv')

for index, row in db_traits_df.iterrows():
    if row['match_id'] not in traits_df['match_id'].values:
        query = "delete from participant_traits where match_id = %s and puuid = %s"
        cur.execute(query, (row['match_id'], row['puuid']))

for index, row in traits_df.iterrows():
    traits_list = eval(row['traits'])
    traits_json = json.dumps(traits_list)
    query = "insert ignore into participant_traits (match_id, puuid, traits) values (%s, %s, %s)"
    cur.execute(query, (row['match_id'], row['puuid'], traits_json))
conn.commit()

In [13]:
cur = conn.cursor(pymysql.cursors.DictCursor)

cur.execute("select match_id, puuid from challenger_match_participants")
db_units_df = pd.DataFrame(cur.fetchall())

units_df = pd.read_csv('challenger_match_participants.csv')

for index, row in db_units_df.iterrows():
    if row['match_id'] not in units_df['match_id'].values:
        query = "delete from participant_units where match_id = %s and puuid = %s"
        cur.execute(query, (row['match_id'], row['puuid']))

for index, row in units_df.iterrows():
    units_list = eval(row['units'])
    units_json = json.dumps(units_list)
    query = "insert ignore into participant_units (match_id, puuid, units) values (%s, %s, %s)"
    cur.execute(query, (row['match_id'], row['puuid'], units_json))
conn.commit()